## 人脸识别

### 概念区分

- **人脸验证** 当我们在火车站让闸机扫描身份证并用脸对着摄像头后，就可以通过闸机了，这就是人脸验证，系统验证了摄像头前面的人和身份证是同一个人。人脸验证是一对一的验证，例如验证摄像头面前的人是否是身份证上的人，又例如验证手机摄像头面前的人是否和之前添加面部识别解锁时留存的照片时同一个人。

- **人脸识别** 人脸识别就是即使没有提供身份证，系统也能识别出摄像头面前这个人是谁。

本次我们将使用一个已经训练好的著名模型——FaceNet，这个模型会将一张图片转换成一个向量，这个向量里包含了 128 个数值，我们通过对比两个向量来判断两张图片是否是同一个人。

In [3]:
# 导入必需的库

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import BatchNormalization, MaxPooling2D, AveragePooling2D, Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import imageio
import os
import numpy as np

from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline

## 将图片编码成 128 维的向量

### 使用卷积网络来进行编码

本次使用的是经过了大量的数据和时间训练的 FaceNet 模型，属于 Inception 网络结果，具体的实现参考同目录下的 `inception_blocks.py`

FaceNet 的输入输出

- 输入图片的像素是 $96\times96$，所以输入是 $(m, n_H, n_W, n_C,) = (m, 96, 96, 3)$
- 输出维度是 $(m, 128)$

In [7]:
# 创建 FactNet 模型
FRmodel = faceRecoModel(input_shape=(96, 96, 3))

# 输出参数数量
print('Total Params:', FRmodel.count_params())

Total Params: 3743280


FaceNet 最后一层是一个全连接层，这一层中有 128 个神经元，所以最后会输入 128 个激活值，这 128 维的向量就代表了输入的图片，也就是将一张图片编码 (encoding) 成了 128 维的向量，我们可以使用这个向量来对比两张面部图片。

<img src="images/distance_kiank.png" style="width:800px;height:250px;">
<caption><center> <u>图 2<br> </u> </center></caption>

判断这个编码优秀应满足下面两个条件：
- 如果两张图片是同一个人，这两张图片的编码非常接近
- 如果两张图片是不同的人，这两张图片的编码相差很远

### 三元组损失

三元组 (triplet) 损失函数可以帮助我们使编码变得优化，这个损失函数会努力让同一个人的两张图片的编码更加接近，同时让不同人的两张图片的编码更加不同。

如果图像用 $x$ 来表示，那么编码可以用 $f(x)$ 来表示，$f(x)$ 就是一个编码函数，这个函数是从神经网络学习得到的。

<img src="images/f_x.png" style="width:450px;height:200px;">

三元组图像组合 $(A, P, N)$:  

- A 是指 **Anchor** 图像，一个人的图像
- P 是指 **Positive** 图像，这张图像里面的人与 A 是同一个人
- N 是指 **Negative** 图像，这张图像里面的人与 A 不是同一个人

也就是说每个训练样本中都包含 3 张图片，所以 $(A^{(i)}, P^{(i)}, N^{(i)})$ 就分别表示第 $i$ 个训练样本的 A,P,N

因为要保证A的编码与P的编码要比A的编码与N的编码更接近，所以就有如下的不等式

通常我们会加一个 $\alpha$ 超参数，它是用来控制这个不等关系的明显程度的:

$$\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2 + \alpha < \mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2$$

因此，下面就是我们的三元组损失函数:

$$\mathcal{J} = \sum^{N}_{i=1} \large[ \small \underbrace{\mid \mid f(A^{(i)}) - f(P^{(i)}) \mid \mid_2^2}_\text{(1)} - \underbrace{\mid \mid f(A^{(i)}) - f(N^{(i)}) \mid \mid_2^2}_\text{(2)} + \alpha \large ] \small_+ \tag{3}$$

上面的公式后面有个小加号，它代表了max()函数，例如 "$[z]_+$" 就等于 $max(z,0)$

解释一下上面的公式:
- 上面公式中标了 (1) 的那一块表示 A 与 P 的编码差异。这个值越小越好。
- 上面公式中标了 (2) 的那一块表示 A 与 N 的编码差异。这个值越大越好。
- $\alpha$ 是一个超参数，是需要我们自己慢慢调参的。在这里我们使用 $\alpha = 0.2$。

In [8]:
# 实现三元组损失函数

def triplet_loss(y_true, y_pred, alpha=0.2):
    '''
    参数：
    y_true -- 暂时不需要的参数
    y_pred -- 包含了三个对象的列表，分别是 A, P, N 的编码

    返回值：
    loss -- 损失值
    '''

    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

    # 计算 A 与 P 的编码差异，即公式中的 (1)
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)))

    # 计算 A 与 N 的编码差异，即公式中的 (2)
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)))

    # 根据两组编码差异来计算损失
    basic_loss = tf.add(tf.subtract(pos_dist, neg_dist), alpha)
    loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)

    return loss

In [9]:
# unit test
tf.random.set_seed(1)
y_true = (None, None, None)
y_pred = (tf.random.normal([3, 128], mean=6, stddev=0.1),
          tf.random.normal([3, 128], mean=1, stddev=1),
          tf.random.normal([3, 128], mean=3, stddev=4))
loss = triplet_loss(y_true, y_pred)
    
tf.print("loss = ", loss)

loss =  427.235168


## 加载模型

我们前面创建了一个 FactNet 模型，但并没有训练，因为需要很多数据和计算力来支持，我们不会选择自己训练它，而是选择加载前人已经训练好的参数值就可以了。

In [10]:
FRmodel.compile(optimizer='adam', loss=triplet_loss, metrics=['accuracy'])
load_weights_from_FaceNet(FRmodel)

## 应用训练好的 FaceNet 模型

### 人脸验证

假设我们的门禁系统是一个人脸验证系统，那么想要进门，就需要出示自己的 ID 卡，并同时将面部对准摄像机，如果系统认为 ID 卡与摄像机面前的是同一个人，那么验证通过，门自动打开。

首先我们得建立一个人脸数据库，这个数据库里面的人都是被允许进门的。数据库里不是存储着人脸图像，而是直接存储着人脸对应的编码向量。这里使用 `img_to_encoding(image_path, model)` 函数，它会利用 FRmodel 模型将输入的图像转换成编码向量，这个函数内部其实就是执行了 FRmodel 模型的一次前向传播，也就是对输入图像进行了一次预测。

这个数据库其实就是个 python 字典，它的 key 是人的名字，value 是这个人对应的人脸编码。

In [12]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

当门口有人刷他的 ID 卡时，就能从 ID 卡中读出他的名字，我们就可以从数据库中调出这个名字对应的人脸编码，然后用这个编码与摄像头拍摄的人脸的编码进行对比，如果差异小于某个阈值，那么就算验证通过，门就会自动为他打开。

In [13]:
# 验证人脸

def verify(image_path, identity, database, model):
    '''
    参数：
    image_path -- 需要被验证的人脸图片地址，也就是摄像头拍到的人脸图片
    identity -- 人名，也就是扫描 ID 后得到的人名
    database -- 人脸数据库
    model -- 前面加载的已经训练好的 FactNet 模型

    返回值：
    dist -- 返回 image_path 的人脸编码和 identity 指定的人名的人脸编码的差异
    door_open -- 验证结果, True/Fasle
    '''

    encoding = img_to_encoding(image_path, model)

    # 对比图片编码差异
    dist = np.linalg.norm(encoding-database[identity])

    # 差异小于 0.7 则表示是同一个人
    # 实际应用中需要更高精度，取比 0.7 更小的阈值
    if dist < 0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
        
    return dist, door_open


使用数据库正确的图片来测试

<img src="images/camera_0.jpg" style="width:100px;height:100px;">

In [14]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

It's younes, welcome home!


(0.6593843, True)

使用错误的图片进行匹配

<img src="images/camera_2.jpg" style="width:100px;height:100px;">

In [15]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(0.8622571, False)

## 人脸识别

上面我们已经实现了人脸验证系统。如果我们的门禁是人脸验证系统，当某人忘记带 ID 卡时，就无法开门了。所以我们下面将门禁升级成人脸识别系统。这样一来，再也不需要 ID 卡了，直接对准摄像头就能识别出此人是否在数据库中并且知道他是谁。当前火车站都还是人脸验证系统，以后会升级成人脸识别系统的，那时我们就不需要带身份证了，我们的脸就是我们的身份证。

In [16]:
# 人脸识别

def who_is_it(image_path, database, model):
    '''
    参数：
    image_path -- 需要被识别的人脸图像的地址
    database -- 有权开门的人脸数据库
    model -- 已经训练好的 FactNet 模型

    Returns:
    min_dist -- 最小差异，我们会一个个与数据库中的人脸编码进行对比，找到与待识别的图像差异最小的那个图像
    identity -- 差异最小的那个人脸图像关联的人名
    '''
    encoding = img_to_encoding(image_path, model)
    min_dist = 100

    for(name, db_enc) in database.items():
        # 对比
        dist = np.linalg.norm(encoding-db_enc)

        if dist < min_dist:
            min_dist = dist
            identity = name
    
    if min_dist > 0.7:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity

In [17]:
# 直接输入 Younes 的图片进行测试
who_is_it("images/camera_0.jpg", database, FRmodel)

it's younes, the distance is 0.6593843


(0.6593843, 'younes')

## 优化方向

- 在数据库中保存一个人的多张照片（同一个人不同光照环境下的多张图片，不同时间段的多张图片等等）。也就是说，拿输入图片与数据库中同一个人多张图像进行验证对比，这样可以提升精准度。

- 将图片中的人脸部分截取出来。也就是说，只对比人脸部分。这样就减少了其它部分，例如衣服，背景等等像素的干扰。这样可以大大的提升精准度。生活中，当我们对准人脸识别摄像头时，屏幕上我们人脸的部分总有一个方框跟随着，它就是在定位并截取我们的人脸部分图像。